In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import string
import re
import nltk
import collections
import tensorflow as tf
import fuzzywuzzy as fuzz

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv


#Import Libraries

In [2]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [3]:
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
ss = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [4]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [6]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (27481, 4)
test shape:  (3534, 3)


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     3534 non-null   object
 1   text       3534 non-null   object
 2   sentiment  3534 non-null   object
dtypes: object(3)
memory usage: 83.0+ KB


In [9]:
train.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [10]:
train.dropna(inplace=True)

In [11]:
train.shape

(27480, 4)

In [12]:
def remove_hyperlinks(text):
    hyperlinkfree=re.sub('https?://\S+|www\.\S+', '', text)
    return hyperlinkfree

def remove(text):
    text=re.sub('\S*\d\S*',' ',text) #Removing Numbers
    text=re.sub('<.*?>+',' ',text)   #Removing Angular Brackets
    text=re.sub('\[.*?\]',' ',text)  #Removing Square Brackets
    text=re.sub('\n',' ',text)       #Removing '\n' character 
    text=re.sub('\*+','',text) #Replacing **** by ABUSE word
    return text

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree


In [13]:
train['text'] = train['text'].apply(lambda x: x.lower())
test['text'] = test['text'].apply(lambda x: x.lower())
train['selected_text'] = train['selected_text'].apply(lambda x: x.lower())

train['text'] = train['text'].apply(lambda x: remove_hyperlinks(x))
test['text'] = test['text'].apply(lambda x: remove_hyperlinks(x))
train['selected_text'] = train['selected_text'].apply(lambda x: remove_hyperlinks(x))

train['text'] = train['text'].apply(lambda x: remove(x))
test['text'] = test['text'].apply(lambda x: remove(x))
train['selected_text'] = train['selected_text'].apply(lambda x: remove(x))

train['text'] = train['text'].apply(lambda x: remove_punctuation(x))
test['text'] = test['text'].apply(lambda x: remove_punctuation(x))
train['selected_text'] = train['selected_text'].apply(lambda x: remove_punctuation(x))

In [14]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,id have responded if i were going,id have responded if i were going,neutral
1,549e992a42,sooo sad i will miss you here in san diego,sooo sad,negative
2,088c60f138,my boss is bullying me,bullying me,negative
3,9642c003ef,what interview leave me alone,leave me alone,negative
4,358bd9e861,sons of why couldnt they put them on the rel...,sons of,negative


In [15]:
train[train['text']==' ']

,textID,text,selected_text,sentiment
1319,bc84f21e3b,,shoesshoesshoesyayyayyayloli,positive
6399,d02b3284fd,,,neutral
8120,4a265d8a34,,,negative
24926,0872ed0f00,,,neutral


In [16]:
train.drop(train[train['text']==' '].index, inplace=True)

In [17]:
def wrong_words(text, selected):
    words = []
    text = text.split()
    selected = selected.split()
    for i in selected:
        if i not in text:
            words.append(i)
    if len(words)>0:
        return " ".join(words)
    else:
        return '++++'

In [18]:
train['spelling'] = train.apply(lambda x: wrong_words(x.text, x.selected_text), axis = 1)

In [19]:
train[train['spelling'].apply(lambda x: len(x)==1)]

,textID,text,selected_text,sentiment,spelling
49,3fcea4debc,which case i got a new one last week and im n...,d im not thrilled at all with mine,negative,d
66,95e12b1cb1,hes awesome have you worked with him before ...,s awesome,positive,s
129,94f67cfa6d,hey mia totally adore your music when will ...,y adore,positive,y
134,6903cb08f2,nice to see you tweeting its sunday may an...,e nice,positive,e
166,c78bf59e67,lichfield tweetup sounds like fun hope to s...,p sounds like fun,positive,p
...,...,...,...,...,...
27153,a044ed928d,enjoy nola definitely one of my favorite citi...,y one of my favorite cities in the world,positive,y
27240,40143b692e,who knows it makes me sad lol,e sad,negative,e
27426,132e051fe8,my cousins moved there like years ago and ...,m sad,negative,m
27470,778184dff1,lol i know and hahadid you fall asleep or ju...,t bored,negative,t


In [20]:
def remove_text(x):
    selected=x[0]
    spelling=x[1]
    selected=selected.split()
    selected.remove(spelling) #https://www.geeksforgeeks.org/python-list-remove/
    return " ".join(selected)

In [21]:
train['selected_text']=train[['selected_text','spelling']].apply(lambda x: remove_text(x) if len(x['spelling'])==1  else x['selected_text'],axis=1)

In [22]:
train['spelling']=train.apply(lambda x: wrong_words(x.text,x.selected_text),axis=1)

In [23]:
train[train['spelling'].apply(lambda x: len(x))==1]

,textID,text,selected_text,sentiment,spelling
